In [1]:
%%capture
!wget https://fisd-dataset.s3.amazonaws.com/fisd-asanti-twi-10p.zip
!unzip /content/twi.zip
!unzip /content/fisd-asanti-twi-10p.zip

In [2]:
%%capture
!pip install librosa
!pip install datasets
!pip install jiwer
!pip install evaluate

In [3]:
import evaluate

# Load the Word Error Rate (WER) metric
metric = evaluate.load("wer")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Using the best model so far.

In [ ]:
!pip install librosa datasets transformers soundfile

import os
import re
import unicodedata
import string
import librosa
import torch
import pandas as pd
import evaluate
from datasets import Dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor

# File paths for model , audio and csv transcriptions
model_checkpoint = "/content/drive/MyDrive/whisper-small-tw/checkpoint-1400"
test_csv_path = "/content/fisd-asanti-twi-10p/data.csv"
audio_base_path = "/content/fisd-asanti-twi-10p/audios"


# Load the processor and model from the checkpoint

processor = WhisperProcessor.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


# Load the test CSV into a pandas DataFrame
test_df = pd.read_csv(test_csv_path, delimiter="\t", on_bad_lines='skip')

# Ensure required columns exist. If your CSV has different column names, change these.

if "Audio Filepath" not in test_df.columns or "Transcription" not in test_df.columns:
    raise ValueError("The test CSV must have 'Audio Filepath' and 'Transcription' columns.")

# Update the audio file paths to full paths if needed
test_df["Audio Filepath"] = test_df["Audio Filepath"].apply(
    lambda x: os.path.join(audio_base_path, os.path.basename(x))
)


# Convert the pandas DataFrame to a Dataset

test_dataset = Dataset.from_pandas(test_df)


# Define a function to load and resample audio
def load_audio(example):
    waveform, sr = librosa.load(example["Audio Filepath"], sr=16000)
    example["audio"] = {
        "array": waveform,
        "sampling_rate": 16000
    }
    return example

# Apply the load_audio function to add "audio" column
test_dataset = test_dataset.map(load_audio, batched=False)


# Define a function to run inference on each sample

def map_to_pred(batch):
    # Process the audio input
    inputs = processor(batch["audio"]["array"], sampling_rate=batch["audio"]["sampling_rate"], return_tensors="pt")
    input_features = inputs["input_features"].to(device)

    # Generate transcription
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            task="transcribe",
            use_cache=True,
            max_length=225
        )
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    batch["prediction"] = transcription
    return batch


# Run inference on the test set

test_pred = test_dataset.map(map_to_pred, batched=False)


# Compute WER
metric = evaluate.load("wer")
test_wer = 100 * metric.compute(predictions=test_pred["prediction"], references=test_pred["Transcription"])
print("Test WER:", test_wer)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


ModuleNotFoundError: No module named 'evaluate'

ValueError: Multiple files found in ZIP file. Only one file per ZIP: ['twi/', 'twi/New Recording 122.m4a', '__MACOSX/twi/._New Recording 122.m4a', 'twi/New Recording 123.m4a', '__MACOSX/twi/._New Recording 123.m4a', 'twi/New Recording 121.m4a', '__MACOSX/twi/._New Recording 121.m4a', 'twi/New Recording 109.m4a', '__MACOSX/twi/._New Recording 109.m4a', 'twi/New Recording 108.m4a', '__MACOSX/twi/._New Recording 108.m4a', 'twi/New Recording 120.m4a', '__MACOSX/twi/._New Recording 120.m4a', 'twi/New Recording 118.m4a', '__MACOSX/twi/._New Recording 118.m4a', 'twi/New Recording 119.m4a', '__MACOSX/twi/._New Recording 119.m4a', 'twi/New Recording 117.m4a', '__MACOSX/twi/._New Recording 117.m4a', 'twi/New Recording 116.m4a', '__MACOSX/twi/._New Recording 116.m4a', 'twi/New Recording 114.m4a', '__MACOSX/twi/._New Recording 114.m4a', 'twi/New Recording 115.m4a', '__MACOSX/twi/._New Recording 115.m4a', 'twi/New Recording 111.m4a', '__MACOSX/twi/._New Recording 111.m4a', 'twi/New Recording 105.m4

In [ ]:
!pip install librosa datasets transformers soundfile

import numpy as np
import os
import re
import unicodedata
import string
import librosa
import torch
import pandas as pd
import evaluate
from datasets import Dataset
from transformers import WhisperForConditionalGeneration, WhisperProcessor

model_checkpoint = "/content/drive/MyDrive/please/checkpoint-1000"
test_csv_path = "/content/fisd-asanti-twi-10p/data.csv"
audio_base_path = "/content/fisd-asanti-twi-10p/audios"

# Load the processor and model from the checkpoint

processor = WhisperProcessor.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Load the test CSV into a pandas DataFrame

import pandas as pd
test_df = pd.read_csv(test_csv_path, delimiter="\t", on_bad_lines='skip')

if "Audio Filepath" not in test_df.columns or "Transcription" not in test_df.columns:
    raise ValueError("The test CSV must have 'Audio Filepath' and 'Transcription' columns.")

# Fix the audio paths
import os
test_df["Audio Filepath"] = test_df["Audio Filepath"].apply(
    lambda x: os.path.join(audio_base_path, os.path.basename(x))
)

metric = evaluate.load("wer")



    # Decode prediction
    predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    # Print result immediately
    print(f"File {i+1}/{len(test_df)}: {audio_path}")
    print("Predicted:", predicted_text)
    print("Reference:", reference_text)
    print("-" * 40)

    # Store for WER calculation
    predictions.append(predicted_text)
    references.append(reference_text)


def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    distance = np.zeros((m+1, n+1), dtype=int)

    for i in range(m+1):
        distance[i][0] = i
    for j in range(n+1):
        distance[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            distance[i][j] = min(
                distance[i-1][j] + 1,
                distance[i][j-1] + 1,
                distance[i-1][j-1] + cost
            )
    return distance[m][n]

def compute_character_error_rate(predictions, references):
    total_distance = 0
    total_chars = 0
    for pred, ref in zip(predictions, references):
        pred = pred.strip()
        ref = ref.strip()
        dist = levenshtein_distance(ref, pred)
        total_distance += dist
        total_chars += len(ref)
    if total_chars == 0:
        return 0.0
    return total_distance / total_chars

predictions = []
references = []

for i, row in test_df.iterrows():
    audio_path = row["Audio Filepath"]
    reference_text = row["Transcription"]

    # Load and resample audio
    waveform, sr = librosa.load(audio_path, sr=16000)

    # Process input features for the model
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"].to(device)

    # Generate prediction
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            task="transcribe",
            use_cache=True,
            max_length=225
        )

    # Decode prediction
    predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
    metric_wer = evaluate.load("wer")
#     # Print result immediately
#     print(f"File {i+1}/{len(test_df)}: {audio_path}")
#     print("Predicted:", predicted_text)
#     print("Reference:", reference_text)
#     print("-" * 40)

#     # Store predictions and references for metric calculation
#     predictions.append(predicted_text)
#     references.append(reference_text)


# Normalize predictions and references
pred_str_norm = [twi_financial_normalizer(p) for p in predictions]
label_str_norm = [twi_financial_normalizer(l) for l in references]

# Filter out empty references to avoid errors in WER computation
filtered_predictions = []
filtered_references = []
for p, r in zip(pred_str_norm, label_str_norm):
    if len(r) > 0:  # Only keep pairs where reference is not empty
        filtered_predictions.append(p)
        filtered_references.append(r)

# Compute WER on normalized text
test_wer = 100 * metric_wer.compute(predictions=filtered_predictions, references=filtered_references)

# Compute CER on normalized text
test_cer = 100 * compute_character_error_rate(filtered_predictions, filtered_references)

print("Final Test WER:", test_wer)
print("Final Test CER:", test_cer)


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


KeyboardInterrupt: 

In [ ]:
import os
import torch
import librosa
import pandas as pd
import evaluate
from transformers import WhisperForConditionalGeneration, WhisperProcessor


model_checkpoint = "/content/drive/MyDrive/final/checkpoint-1000"
test_csv_path = "/content/fisd-asanti-twi-10p/data.csv"
audio_base_path = "/content/fisd-asanti-twi-10p/audios"

# Load the processor and model from the checkpoint
processor = WhisperProcessor.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Load the test CSV into a pandas DataFrame
test_df = pd.read_csv(test_csv_path, delimiter="\t", on_bad_lines='skip')
if "Audio Filepath" not in test_df.columns or "Transcription" not in test_df.columns:
    raise ValueError("The test CSV must have 'Audio Filepath' and 'Transcription' columns.")

# Fix the audio paths
test_df["Audio Filepath"] = test_df["Audio Filepath"].apply(
    lambda x: os.path.join(audio_base_path, os.path.basename(x))
)

# Metrics and Normalizer Setup
metric_wer = evaluate.load("wer")

def twi_financial_normalizer(text: str) -> str:
    # Replace this with your actual normalization logic
    text = text.lower().strip()
    return text

def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    import numpy as np
    distance = np.zeros((m+1, n+1), dtype=int)

    for i in range(m+1):
        distance[i][0] = i
    for j in range(n+1):
        distance[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            distance[i][j] = min(
                distance[i-1][j] + 1,
                distance[i][j-1] + 1,
                distance[i-1][j-1] + cost
            )
    return distance[m][n]

def compute_character_error_rate(predictions, references):
    total_distance = 0
    total_chars = 0
    for pred, ref in zip(predictions, references):
        pred = pred.strip()
        ref = ref.strip()
        dist = levenshtein_distance(ref, pred)
        total_distance += dist
        total_chars += len(ref)
    if total_chars == 0:
        return 0.0
    return total_distance / total_chars

# Inference Loop
predictions = []
references = []

for i, row in test_df.iterrows():
    audio_path = row["Audio Filepath"]
    reference_text = row["Transcription"]

    # Load and resample audio
    waveform, sr = librosa.load(audio_path, sr=16000)

    # Process input features for the model
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"].to(device)

    # Generate prediction
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            task="transcribe",
            use_cache=True,
            max_length=225
        )

    # Decode prediction
    predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    # Print result immediately (optional)
    print(f"File {i+1}/{len(test_df)}: {audio_path}")
    print("Predicted:", predicted_text)
    print("Reference:", reference_text)
    print("-" * 40)

    # Store for metric calculation
    predictions.append(predicted_text)
    references.append(reference_text)

##########################################
# Compute WER and CER
##########################################
# Normalize predictions and references
pred_str_norm = [twi_financial_normalizer(p) for p in predictions]
label_str_norm = [twi_financial_normalizer(l) for l in references]

# Filter out empty references to avoid errors in WER computation
filtered_predictions = []
filtered_references = []
for p, r in zip(pred_str_norm, label_str_norm):
    if len(r) > 0:  # Only keep pairs where reference is not empty
        filtered_predictions.append(p)
        filtered_references.append(r)

# Compute WER on normalized text
wer = 100 * metric_wer.compute(predictions=filtered_predictions, references=filtered_references)

# Compute CER on normalized text
cer = 100 * compute_character_error_rate(filtered_predictions, filtered_references)

print("Final Test WER:", wer)
print("Final Test CER:", cer)


File 1/3187: /content/fisd-asanti-twi-10p/audios/AsantiTwiMa25-SItRBDya-Tmp023-L8PkrD.ogg
Predicted: MAA WƆN HO YƐƐ HUAM
Reference: Maa wɔn ho yɛɛ huam
----------------------------------------
File 2/3187: /content/fisd-asanti-twi-10p/audios/AsantiTwiMa20-SLWOah2f-Tmp023-GXRJsS.ogg
Predicted: MAA WƆN HO YƐƐ HUAM
Reference: Maa wɔn ho yɛɛ huam
----------------------------------------
File 3/3187: /content/fisd-asanti-twi-10p/audios/AsantiTwiMa25-JART851s-Tmp023-S0dsgA.ogg
Predicted: MAA WƆN HO YƐƐ HUAM
Reference: Maa wɔn ho yɛɛ huam
----------------------------------------
File 4/3187: /content/fisd-asanti-twi-10p/audios/AsantiTwiFm58-GA6XjFAL-Tmp023-pTurw5.ogg
Predicted: MAA WƆN HO YƐƐ HUAM
Reference: Maa wɔn ho yɛɛ huam
----------------------------------------
File 5/3187: /content/fisd-asanti-twi-10p/audios/AsantiTwiFm24-STWjCZCq-Tmp023-aGXu29.ogg
Predicted: MAA WƆN HO YƐƐ HUAM
Reference: Maa wɔn ho yɛɛ huam
----------------------------------------
File 6/3187: /content/fisd-asanti-t

In [9]:
import os
import torch
import librosa
import pandas as pd
import evaluate
from transformers import WhisperForConditionalGeneration, WhisperProcessor

##########################################
# User-defined paths (adjust these)
##########################################
model_checkpoint = "/content/drive/MyDrive/final/checkpoint-1000"  # e.g. "/content/drive/MyDrive/whisper-small-tw"
test_csv_path = "/content/data.xlsx"          # e.g. "/content/test_data.csv"
audio_base_path = "/content/fisd-asanti-twi-10p/audios"               # e.g. "/content/test_audios"

##########################################
# Load the processor and model from the checkpoint
##########################################
processor = WhisperProcessor.from_pretrained(model_checkpoint)
model = WhisperForConditionalGeneration.from_pretrained(model_checkpoint)
model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

##########################################
# Load the test CSV into a pandas DataFrame
##########################################
test_df = pd.read_excel(test_csv_path)
if "Audio Filepath" not in test_df.columns or "Transcription" not in test_df.columns:
    raise ValueError("The test CSV must have 'Audio Filepath' and 'Transcription' columns.")

# Fix the audio paths
test_df["Audio Filepath"] = test_df["Audio Filepath"].apply(
    lambda x: os.path.join(audio_base_path, os.path.basename(x))
)

##########################################
# Metrics and Normalizer Setup
##########################################
metric_wer = evaluate.load("wer")

def twi_financial_normalizer(text: str) -> str:
    # Replace this with your actual normalization logic
    text = text.lower().strip()
    return text

def levenshtein_distance(s1, s2):
    m, n = len(s1), len(s2)
    import numpy as np
    distance = np.zeros((m+1, n+1), dtype=int)

    for i in range(m+1):
        distance[i][0] = i
    for j in range(n+1):
        distance[0][j] = j

    for i in range(1, m+1):
        for j in range(1, n+1):
            cost = 0 if s1[i-1] == s2[j-1] else 1
            distance[i][j] = min(
                distance[i-1][j] + 1,
                distance[i][j-1] + 1,
                distance[i-1][j-1] + cost
            )
    return distance[m][n]

def compute_character_error_rate(predictions, references):
    total_distance = 0
    total_chars = 0
    for pred, ref in zip(predictions, references):
        pred = pred.strip()
        ref = ref.strip()
        dist = levenshtein_distance(ref, pred)
        total_distance += dist
        total_chars += len(ref)
    if total_chars == 0:
        return 0.0
    return total_distance / total_chars

##########################################
# Inference Loop
##########################################
predictions = []
references = []

for i, row in test_df.iterrows():
    audio_path = row["Audio Filepath"]
    reference_text = row["Transcription"]

    # Load and resample audio
    waveform, sr = librosa.load(audio_path, sr=16000)

    # Process input features for the model
    inputs = processor(waveform, sampling_rate=16000, return_tensors="pt")
    input_features = inputs["input_features"].to(device)

    # Generate prediction
    with torch.no_grad():
        predicted_ids = model.generate(
            input_features,
            task="transcribe",
            use_cache=True,
            max_length=225
        )

    # Decode prediction
    predicted_text = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

    # Print result immediately (optional)
    print(f"File {i+1}/{len(test_df)}: {audio_path}")
    print("Predicted:", predicted_text)
    print("Reference:", reference_text)
    print("-" * 40)

    # Store for metric calculation
    predictions.append(predicted_text)
    references.append(reference_text)

##########################################
# Compute WER and CER
##########################################
# Normalize predictions and references
pred_str_norm = [twi_financial_normalizer(p) for p in predictions]
label_str_norm = [twi_financial_normalizer(l) for l in references]

# Filter out empty references to avoid errors in WER computation
filtered_predictions = []
filtered_references = []
for p, r in zip(pred_str_norm, label_str_norm):
    if len(r) > 0:  # Only keep pairs where reference is not empty
        filtered_predictions.append(p)
        filtered_references.append(r)

# Compute WER on normalized text
wer = 100 * metric_wer.compute(predictions=filtered_predictions, references=filtered_references)

# Compute CER on normalized text
cer = 100 * compute_character_error_rate(filtered_predictions, filtered_references)

print("Final Test WER:", wer)
print("Final Test CER:", cer)


<ipython-input-9-b8eede3389f3>:91: UserWarning: PySoundFile failed. Trying audioread instead.
  waveform, sr = librosa.load(audio_path, sr=16000)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


FileNotFoundError: [Errno 2] No such file or directory: '/content/fisd-asanti-twi-10p/audios/New Recording 104.m4a'